# 李宏毅老师homework1 [原文代码](https://colab.research.google.com/drive/131sSqmrmWXfjFZ3jWSELl8cm0Ox5ah3C)
# 1. 数据的读取

## 1.1 导入相应的包裤pandas和numpy

In [ ]:
import pandas as pd
import numpy as np

## 1.2 读取数据并且将原始数据中“NR”替换为0，将读取的数据（dataframe) 转化为(np.array)

In [ ]:
data=pd.read_csv("/homework1/homework1.csv")#文件的读取，填写好你数据文件的位置
data[data == 'NR'] = 0 #替换不是数字型的数据，将其改为0
data


In [ ]:
raw_data = data.to_numpy()#因为原始数据是csv,然后用了pandas里的dataframe,现在这一步是为了将dataframe转化为np.array
raw_data


## 1.3 为了方便起见，我只用了2个月的数据（原来课件是包含12个月数据)，大家按自己的需求适当调整下代码即可。比如：你要12个月的数据，下文中的2将改为12即可

In [ ]:
print(data.shape)
print(raw_data.shape)
#这里打印出来是为了提醒大家我读取的数据是2个月的，720行=18*20*2=（18个features)*(每个月20天)*（总共2个月）
#26列 = 每天24个小时+加上前面的2列（Dates,Features) 

# 2. 数据预处理
## 处理之后的training data 的数据结构长啥样，请看这个草图：[train_data](https://github.com/Appdevelophao/Python-Basics-Learning/blob/master/train_data_structure.jpeg)

## 说一下数据处理的基本思路和路线：具体长啥样请点击：[raw_sample_month](https://github.com/Appdevelophao/Python-Basics-Learning/blob/master/Lee_ML_homework1/Raw_smaple_month.jpeg)
- Step1: sample 
- Step2: month_data
- Step3: train_data (包括两个部分：一个是features， 另一个是labels)

## 2.1 Sample and Month_data

In [ ]:
month_data =np.empty(shape=[18,480*2])#因为我这里只用了2个月的数据，所以预设的month_data的数据结构为： 18行, 480*2列。18为18个features，480=24小时*20天，2为2个月 
for month in range(2):
    sample=np.empty([18,480])#sample只起到中间转换的作用，sample其实就是一个月（第1st,2nd,...) 的数据,然后month_data就是将每个月（sample)的数据进行按列叠加
    for day in range(20):
        sample[:, day * 24: (day + 1) * 24] = raw_data[18 * (20 * month + day): 18 * (20 * month + day + 1),2:]#一天24个小时
        #当你看不懂的这些抽象的数据表达时，你可以赋值带入具体的数字进去看看，比如：当month=0,day=0, raw_data[0:18,2:1]就是原始数据的前18行，第3列到最后一列（前两列是dates 和 features)
        #sample[:,0:24]是一个18行，24列的np.array(一天的数据量)
        #固定好月数， 每月20天，所以每月叠加20次， 又因为我用了2个月的数据，所以要叠加2*20次。每一个叠加单位（一天的数据）是一个18*24维度的数据
    print(sample.shape)
    
    
    month_data[:,480*month:480*(month+1)] = sample
print(month_data.shape)

month_data


## 2.2 train_data (x:Features, y:Labels)

In [ ]:
x=np.empty([2*471,18*9],dtype=float)#这是最终的训练集的features,因为每个月只有20天，每天24个小时，而且每个月的最后9个小时是用不上的，所以每个月的数据条为20*24-9=471个 
y=np.empty([2*471,1],dtype=float) #（针对于每个特定的数据条（features), 对应于这个数据条的label就是该数据条往后推一个小时的PM2.5的值
for month in range(2):
    for day in range(20):
        for hour in range(24):#想象一下：以18行9列的数据框从开始获取数据，然后每框取一次数据，该数据框就向后挪动一单位继续框取数据，在每个月内，共计框取471次
            if day==19 and hour >14:#这个的意思是：每个月的第19天第14点到23点的数据是不需要的（每个月的最后9小时）
                continue
            x[month*471+day*24+hour,:]=month_data[:,480*month + day * 24 + hour : 480*month+day * 24 + hour + 9].reshape(1, -1)
            #这里的reshape大家可以参考下数据具体长啥样子的图片， 其实本质上就是将诸多行拉长为一列， 从而形成一个数据条
            y[month * 471 + day * 24 + hour, 0] = month_data[9, 480*month+day * 24 + hour + 9]   
            
            
print('x\' shape is ', x.shape)
print('y\' shape is ', y.shape)
print(x)
print(y)
len(x[0])


## 2.3 数据的归一化处理 Normalization 

In [ ]:
mean_x = np.mean(x, axis = 0) # 沿着每行的行方向求均值
std_x = np.std(x, axis = 0) # 沿着每行的行方向求标准差
for i in range(len(x)): #2 * 471 行数
    for j in range(len(x[0])): #18 * 9 列数
        if std_x[j] != 0: #标准差不等于0
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x.shape

## 2.4将训练集划分为测试集（train_set）和 验证集（validationion set ), 比例为 8:2

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]


# 3. Regression 核心部分

## 3.1 设置一些linear Regression的参数初始值

In [ ]:
dim = 18 * 9 + 1 #这其实就是Regression的参数的维数，每个样本数据有18*9个features,再加上一个常数项b,即为：w_1,w_2,..., w_162,b 
w = np.ones([dim, 1])# w即为参数集合，然后这里的意思是默认为参数的初始值全部设置为1，因次w为163行 1列
x = np.concatenate((np.ones([2 * 471, 1]), x), axis = 1).astype(float)
#这个就是就在原来 x 的基础上，在 x的右边加了一列，所以加完之后的 x的shape为：942行*163列


adagrad = np.zeros([dim, 1])
learning_rate = 100  # 学习率
iter_time = 10000  # 迭代次数
eps = 0.0000000001 #一个很小的数

print(x.shape)
print(w.shape)

In [ ]:
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/2)#计算损失函数:w_1*F_1+w_2*F_2+,...,w_162*F_162+b
    if(t%100==0): #每100次打印一次损失函数
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
    np.save('weight.npy', w)
w #w即为通过样本迭代了很多次计算出来的参数：w_1，w_2, ...,w_162,b

In [ ]:
w.shape

## 3.2 用validation的数据看看上面训练出来参数是不是OK
- 其实熟悉的人都知道这样不合理，因为我们上面用了全部的训练集（包括了validation的数据）来训练模型，现在用再用validation来验证，这样的话，闭着验证都知道，验证的结果肯定比较好

In [ ]:
w = np.load('weight.npy')
# 使用x_validation和y_validation来计算模型的准确率，因为X已经normalize了，所以不需要再来一遍，只需在x_validation上添加新的一列作为bias的乘数即可
x_validation = np.concatenate((np.ones([189, 1]), x_validation), axis=1).astype(float)
ans_y = np.dot(x_validation, w)
loss = np.sqrt(np.sum(np.power(ans_y - y_validation, 2)) / 1131)
print(loss)

## 3.3 处理下测试集test_data的数据
- 我只用了6个测试样本, 大家只需要按照自己的需要选取测试样本的个数，注意：每个测试样本是一个18*9的数据

In [ ]:
testdata=pd.read_csv("/homework1/homework1_test.csv")#读取test_data
test_data = testdata.iloc[:,2:] #切片，选择数字的部分
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy() 
test_data
test_x = np.empty([6, 18 * 9], dtype=float)
for i in range(6):  # 共240个测试输入数据
    test_x[i, :] = test_data[18 * i: 18 * (i + 1), :].reshape(1, -1)# 将每18行拉长变为一行，从而形成一个数据条

## 3.4 测试数据的归一标准化（和前面的几乎一样）

In [ ]:
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([6, 1]), test_x), axis=1).astype(float)

## 3.5 利用模型完成预测 （预测工作到此结束）恭喜你

In [ ]:
# 进行预测 
w = np.load('weight.npy')
ans_y = np.dot(test_x, w) 
print(ans_y)


# 4 将预测结果保存在自动生成的名为submit.csv 的文件里

In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(6):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)